In [53]:
import pandas as pd
import biblib.bib
import re
import graphviz
import os
import sys
import math
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz
import matplotlib.pyplot as plt
from semanticscholar import SemanticScholar
from unidecode import unidecode
from alphabet_detector import AlphabetDetector
import seaborn as sns
from netgraph import Graph
import nltk
from nltk.stem import WordNetLemmatizer
import collections
import re
import json

nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samueleceol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/samueleceol/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/samueleceol/nltk_data...


True

In [2]:
# Glossary file contains:
    # All paper from initial selection (65)
    # All the FS and BS papers obtained from the query with proximity constraint
def get_glossary():
    with open('Glossary.bib', 'r') as glossary:
        db = biblib.bib.Parser().parse(glossary, log_fp=sys.stderr).get_entries()
        return db
glossary = get_glossary()

In [3]:
components = list(nx.connected_components(nx.Graph(nx.nx_pydot.read_dot('ALL_selection_only.dot'))))
components_info = []

for component in components:
    cur_component = {}

    for paper in component:
        cur_component[paper] = {}

        if paper.replace('\\n','') in glossary:
            cur_paper = glossary[paper.replace('\\n','')]
        else:
            print(f"Paper not in Glossary: {paper}")
        cur_component[paper]['title'] = cur_paper['title']
        cur_component[paper]['author'] = cur_paper['author']
        if('journal' in cur_paper):
            cur_component[paper]['journal'] = cur_paper['journal']
        else:
            print(f"missing journal: {paper}")
        if('abstract' in cur_paper):
            cur_component[paper]['abstract'] = cur_paper['abstract']
        else:
            print(f"missing abstract: {paper}")

    components_info.append(cur_component)

Paper not in Glossary: \n


In [58]:
title_abstr_str = ''
lemmatizer = WordNetLemmatizer()

# 0, 2, 8
cur_comp_info = components_info[0]
for paper in cur_comp_info:
    cur_paper = cur_comp_info[paper]
    title_abstr_str += f" {cur_paper['title']}"
    title_abstr_str += f" {cur_paper['abstract']}"

def remove_punctuation(input_string):
    return re.sub(r'[^\w\s]', '', input_string)

def remove_stopwords(input_string):
    tokens = nltk.word_tokenize(input_string.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    bigrams_trigrams = list(nltk.everygrams(tokens, 2, 3))
    bigrams_trigrams = ['_'.join(list(gram)).replace("'", '') for gram in bigrams_trigrams]
    tokens += bigrams_trigrams
    return tokens


def count_tokens(input_string):
    tokens = remove_stopwords(remove_punctuation(input_string))
    return collections.Counter(tokens)

token_count = count_tokens(title_abstr_str)
sorted_token_count = dict(sorted(token_count.items(), key=lambda item: item[1], reverse=True))

print(sorted_token_count)

{'topic': 272, 'model': 80, 'label': 65, 'topic_model': 34, 'using': 33, 'labeling': 33, 'research': 30, 'approach': 27, 'term': 24, 'method': 24, 'document': 21, 'automatic': 21, 'measure': 21, 'medium': 20, 'coherence': 20, 'topic_labeling': 20, 'topic_label': 19, 'list': 18, 'human': 18, 'word': 17, 'public': 17, 'news': 17, 'social': 16, 'modeling': 15, 'text': 15, 'quality': 15, 'vaccine': 15, 'result': 14, 'sentence': 14, 'lda': 14, 'representation': 13, 'different': 13, 'information': 13, 'evaluation': 13, 'used': 13, 'analysis': 13, 'neural': 13, 'interpretability': 13, 'topic_modeling': 13, 'latent': 12, 'work': 12, 'paper': 12, 'study': 12, 'two': 12, 'image': 11, 'also': 11, 'network': 11, 'applied': 11, 'propose': 11, 'trend': 11, 'growth': 11, 'automatic_topic': 11, 'collection': 10, 'task': 10, 'language': 10, 'novel': 10, 'generated': 10, 'twitter': 10, 'community': 10, 'way': 9, 'system': 9, 'textual': 9, 'article': 9, 'based': 9, 'one': 9, 'show': 9, 'present': 9, 'rec

In [59]:
for paper in cur_comp_info:
    print(cur_comp_info[paper]['title'])

Evaluating Topic Representations for Exploring Document Collections
Automatic Generation of Topic Labels
Moving beyond word lists: towards abstractive topic labels for human-like topics of scientific documents
{TLATR}: {Automatic} {Topic} {Labeling} {Using} {Automatic} ({Domain}-{Specific}) {Term} {Recognition}
An Ontology-Based Labeling of Influential Topics Using Topic Network Analysis
Labeling Topics with Images Using a Neural Network
Automatic {Labeling} of {Topic} {Models} {Using} {Graph}-{Based} {Ranking}
Characterizing diabetes, diet, exercise, and obesity comments on Twitter
Multimodal {Topic} {Labelling}
Automatic {Topic} {Labeling} model with {Paired}-{Attention} based on {Pre}-trained {Deep} {Neural} {Network}
Topically Driven Neural Language Model
Community Topic: Topic Model Inference by Consecutive Word Community Discovery
Experimental explorations on short text topic mining between LDA and NMF based Schemes
Automatic topic labeling using graph-based pre-trained neural em